# Squats

In [19]:
import cv2
from cv2 import destroyAllWindows
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [20]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

In [21]:
def rescale_frame(frame, percent=50):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [28]:
# Youtube video
angle_min = []
angle_min_hip = []
cap = cv2.VideoCapture(0)

print("Debug - 1")

# Curl counter variables
counter = 0 
min_ang = 0
max_ang = 0
min_ang_hip = 0
max_ang_hip = 0
stage = None

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size = (640, 480)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('output_video_.mp4', fourcc, 24, size)

print("Debut - 2")

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        print("Debut - 3")
        ret, frame = cap.read()
        if frame is not None:
            frame_ = rescale_frame(frame, percent=75)
        
        print("Debug - 4")
        # Recolor image to RGB
        image = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            
            print("Debug - 5")
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            
            # Get coordinates
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            
           
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            
            angle_knee = calculate_angle(hip, knee, ankle) #Knee joint angle
            angle_knee = round(angle_knee,2)
            
            angle_hip = calculate_angle(shoulder, hip, knee)
            angle_hip = round(angle_hip,2)
            
            hip_angle = 180-angle_hip
            knee_angle = 180-angle_knee
            
            print(angle_knee, angle_hip)
            
            angle_min.append(angle_knee)
            angle_min_hip.append(angle_hip)
            
            print(angle_knee)
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
                       
                
            cv2.putText(image, str(angle_knee), 
                           tuple(np.multiply(knee, [1500, 800]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA
                                )
            
            cv2.putText(image, str(angle_hip), 
                           tuple(np.multiply(hip, [1500, 800]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            print("Debug - 6")
            # Curl counter logic
            if angle_knee > 169:
                stage = "up"
            if angle_knee <= 90 and stage =='up':
                stage="down"
                counter +=1
                print(counter)
                print(counter)
                min_ang  =min(angle_min)
                max_ang = max(angle_min)
                
                min_ang_hip  =min(angle_min_hip)
                max_ang_hip = max(angle_min_hip)
                
                print(min(angle_min), " _ ", max(angle_min))
                print(min(angle_min_hip), " _ ", max(angle_min_hip))
                angle_min = []
                angle_min_hip = []
        except:
            pass
        
        # Render squat counter
        # Setup status box
        cv2.rectangle(image, (20,20), (435,160), (0,0,0), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
        cv2.putText(image, "Repetition : " + str(counter), 
                    (30,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (10,120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        #Knee angle:
        cv2.putText(image, 'Angle', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
        cv2.putText(image, "Knee-joint angle : " + str(min_ang), 
                    (30,100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
        #Hip angle:
        cv2.putText(image, "Hip-joint angle : " + str(min_ang_hip), 
                    (30,140), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        

        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(203,17,17), thickness=2, circle_radius=2) 
                                 )               
        
        out.write(image)
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            cap.release()
            out.release()
            cv2.destroyAllWindows()
            #break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

Debug - 1
Debut - 2
Debut - 3


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Debug - 4
Debug - 5
Debut - 3
Debug - 4
Debug - 5
Debut - 3
Debug - 4
Debug - 5
179.8 177.11
179.8
Debug - 6
Debut - 3
Debug - 4
Debug - 5
179.1 176.91
179.1
Debug - 6
Debut - 3
Debug - 4
Debug - 5
177.77 176.03
177.77
Debug - 6
Debut - 3
Debug - 4
Debug - 5
177.55 175.51
177.55
Debug - 6
Debut - 3
Debug - 4
Debug - 5
174.35 174.42
174.35
Debug - 6
Debut - 3
Debug - 4
Debug - 5
176.5 175.36
176.5
Debug - 6
Debut - 3
Debug - 4
Debug - 5
176.44 175.53
176.44
Debug - 6
Debut - 3
Debug - 4
Debug - 5
176.53 175.44
176.53
Debug - 6
Debut - 3
Debug - 4
Debug - 5
177.28 175.56
177.28
Debug - 6
Debut - 3
Debug - 4
Debug - 5
177.54 175.88
177.54
Debug - 6
Debut - 3
Debug - 4
Debug - 5
179.35 176.91
179.35
Debug - 6
Debut - 3
Debug - 4
Debug - 5
178.71 176.22
178.71
Debug - 6
Debut - 3
Debug - 4
Debug - 5
176.66 177.85
176.66
Debug - 6
Debut - 3
Debug - 4
Debug - 5
17.32 13.88
17.32
Debug - 6
1
1
17.32  _  179.8
13.88  _  177.85
Debut - 3
Debug - 4
Debug - 5
14.76 11.52
14.76
Debug - 6
Debut - 3


Debut - 3
Debug - 4
Debug - 5
180.0 172.08
180.0
Debug - 6
Debut - 3
Debug - 4
Debug - 5
177.94 171.2
177.94
Debug - 6
Debut - 3
Debug - 4
Debug - 5
176.61 171.3
176.61
Debug - 6
Debut - 3
Debug - 4
Debug - 5
177.78 170.79
177.78
Debug - 6
Debut - 3
Debug - 4
Debug - 5
178.64 171.96
178.64
Debug - 6
Debut - 3
Debug - 4
Debug - 5
178.63 171.19
178.63
Debug - 6
Debut - 3
Debug - 4
Debug - 5
178.89 171.62
178.89
Debug - 6
Debut - 3
Debug - 4
Debug - 5
179.49 173.23
179.49
Debug - 6
Debut - 3
Debug - 4
Debug - 5
178.66 172.69
178.66
Debug - 6
Debut - 3
Debug - 4
Debug - 5
178.32 172.06
178.32
Debug - 6
Debut - 3
Debug - 4
Debug - 5
178.79 172.13
178.79
Debug - 6
Debut - 3
Debug - 4
Debug - 5
178.46 172.02
178.46
Debug - 6
Debut - 3
Debug - 4
Debug - 5
177.36 171.27
177.36
Debug - 6
Debut - 3
Debug - 4
Debug - 5
177.05 169.92
177.05
Debug - 6
Debut - 3
Debug - 4
Debug - 5
174.41 167.73
174.41
Debug - 6
Debut - 3
Debug - 4
Debug - 5
170.64 166.86
170.64
Debug - 6
Debut - 3
Debug - 4
Debug - 

Debug - 5
179.7 176.81
179.7
Debug - 6
Debut - 3
Debug - 4
Debug - 5
176.9 178.48
176.9
Debug - 6
Debut - 3
Debug - 4
Debug - 5
172.47 179.27
172.47
Debug - 6
Debut - 3
Debug - 4
Debug - 5
167.46 178.17
167.46
Debug - 6
Debut - 3
Debug - 4
Debug - 5
163.79 175.68
163.79
Debug - 6
Debut - 3
Debug - 4
Debug - 5
163.48 173.69
163.48
Debug - 6
Debut - 3
Debug - 4
Debug - 5
165.7 172.81
165.7
Debug - 6
Debut - 3
Debug - 4
Debug - 5
168.9 170.22
168.9
Debug - 6
Debut - 3
Debug - 4
Debug - 5
172.96 169.26
172.96
Debug - 6
Debut - 3
Debug - 4
Debug - 5
171.42 165.3
171.42
Debug - 6
Debut - 3
Debug - 4
Debug - 5
170.09 159.11
170.09
Debug - 6
Debut - 3
Debug - 4
Debug - 5
165.48 152.9
165.48
Debug - 6
Debut - 3
Debug - 4
Debug - 5
158.68 146.38
158.68
Debug - 6
Debut - 3
Debug - 4
Debug - 5
158.59 143.8
158.59
Debug - 6
Debut - 3
Debug - 4
Debug - 5
154.2 139.59
154.2
Debug - 6
Debut - 3
Debug - 4
Debug - 5
146.37 132.21
146.37
Debug - 6
Debut - 3
Debug - 4
Debug - 5
142.61 127.85
142.61
Debug 

Debug - 4
Debug - 5
61.23 0.79
61.23
Debug - 6
7
7
11.08  _  177.95
0.29  _  163.91
Debut - 3
Debug - 4
Debug - 5
139.08 33.7
139.08
Debug - 6
Debut - 3
Debug - 4
Debug - 5
99.71 9.73
99.71
Debug - 6
Debut - 3
Debug - 4
Debug - 5
156.22 0.94
156.22
Debug - 6
Debut - 3
Debug - 4
Debug - 5
179.46 1.85
179.46
Debug - 6
Debut - 3
Debug - 4
Debug - 5
172.6 8.24
172.6
Debug - 6
Debut - 3
Debug - 4
Debug - 5
163.45 2.75
163.45
Debug - 6
Debut - 3
Debug - 4
Debug - 5
174.56 6.41
174.56
Debug - 6
Debut - 3
Debug - 4
Debug - 5
78.09 2.65
78.09
Debug - 6
8
8
78.09  _  179.46
0.94  _  33.7
Debut - 3
Debug - 4
Debug - 5
17.77 22.91
17.77
Debug - 6
Debut - 3
Debug - 4
Debug - 5
169.35 3.96
169.35
Debug - 6
Debut - 3
Debug - 4
Debug - 5
170.17 3.72
170.17
Debug - 6
Debut - 3
Debug - 4
Debug - 5
170.2 2.65
170.2
Debug - 6
Debut - 3
Debug - 4
Debug - 5
173.6 1.81
173.6
Debug - 6
Debut - 3
Debug - 4
Debug - 5
171.5 2.84
171.5
Debug - 6
Debut - 3
Debug - 4
Debug - 5
167.25 0.84
167.25
Debug - 6
Debut - 3

# Knee

In [23]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [29]:
cap = cv2.VideoCapture("KneeBendVideo.mp4")
# KneeBendVideo.mp4
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

import time
start = time.time()

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )     
        
        hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
        ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
        
        angle = str(round(calculate_angle(hip, knee, ankle)))
        text_for_angle = f"Angle of Knee Bend: {angle}"
        
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(image, text_for_angle, (400,30), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()